# HW7: Beam Search Decoding - News Headline Generation

In this exercise, you are going to learn and implement decoding techniques for sequence generation. Usually, the sequence is generated word-by-word from a model. In each step, the model predicted the most likely word based on the predicted words in previous steps (this is called auto-regressive decoding).

As such, it is very important how you decide on what to predicted at each step, as it will be conditioned on to predicted all of the following steps. We will implement two of main decoding techniques introduced in the lecture: **Greedy Decoding** and **Beam Search Decoding**. Greedy Decoding immediately chooses the word with best score at each step, while Beam Search Decoding focuses on the sequence that give the best score overall.

To complete this exercise, you will need to complete the methods for decoding for a text generation model trained on [New York Times Comments and Headlines dataset](https://www.kaggle.com/aashita/nyt-comments). The model is trained to predict a headline for the news given seed text. You do not need to train any model model in this exercise as we provide both the pretrained model and dictionary.


## Download model and vocab and setup

In [7]:
!wget -O vocab.txt "https://www.dropbox.com/scl/fi/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0"
!wget -O model.bin "https://www.dropbox.com/s/okmri7cnd729rr5/hw9_model.bin?dl=0"

zsh:1: command not found: wget
zsh:1: command not found: wget


In [2]:
import torch
import torch.nn as nn
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tqdm import tqdm
import math
import numpy as np

In [3]:
class RNNmodel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_rate):

        super().__init__()
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, 128, num_layers=2,
                     batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, vocab_size)

    def forward(self, src):
        embedding = self.embedding(src)
        output,_ = self.rnn(embedding)
        output = self.dropout(output)
        prediction = self.fc2(output)
        return prediction

In [4]:
with open("vocab.txt") as f:
  vocab_file = f.readlines()
embedding_dim = 64
dropout_rate = 0.2

model = RNNmodel(len(vocab_file), embedding_dim, dropout_rate)
model.load_state_dict(torch.load("model.bin",map_location='cpu'))
model.eval()

/var/folders/27/0_8pjdb52g91gmnzbtgblvgc0000gn/T/ipykernel_17063/3839689483.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.bin",

RNNmodel(
  (embedding): Embedding(10054, 64)
  (rnn): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=10054, bias=True)
)

In [5]:
vocab = [v.strip() for v in vocab_file]
vocab_size = len(vocab)
print(f"Vocab Size: {vocab_size}")
vocab[:10]

Vocab Size: 10054


['<unk>', '<pad>', '<eos>', 'the', 'a', 'to', 'of', 's', 'in', 'for']

In [6]:
stoi = { ch:i for i,ch in enumerate(vocab) }
tokenizer = Tokenizer(WordLevel(stoi, unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
tokenized_text = tokenizer.encode("the a of to unknowns")
print(tokenized_text)
print(tokenized_text.ids)
print(tokenized_text.tokens)
print(tokenizer.decode(tokenized_text.ids))

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[3, 4, 6, 5, 0]
['the', 'a', 'of', 'to', '<unk>']
the a of to <unk>


In [7]:
tokenized_text.ids
# tokenizer.decode(tokenized_text.ids)

[3, 4, 6, 5, 0]

In [8]:
ids = torch.tensor(tokenized_text.ids).unsqueeze(0)
print(ids)
result = model(ids)
print(result.shape)

# pass the result tensor to softmax to get the probabilities
probs = nn.functional.softmax(result[0], dim=-1)
print(probs.shape)

# get the top 5 probabilities
top5_probs, top5_token_ids = torch.topk(probs, 5)
print(top5_probs.shape, top5_token_ids.shape)

# convert the top 5 token ids to tokens
# top5_tokens = [vocab[i] for i in top5_token_ids[-1]]
top5_tokens = tokenizer.decode(top5_token_ids[-1].tolist())
print(top5_tokens)

tensor([[3, 4, 6, 5, 0]])
torch.Size([1, 5, 10054])
torch.Size([5, 10054])
torch.Size([5, 5]) torch.Size([5, 5])
of and for on in


## 1. TODO: Greedy decode
Normally, in sequence generation task, the model will continue generating tokens until an end-of-sequence symbol appear or the maximum length is reached. For this task:
- The end-of-sequence symbol is "< eos >" and its index is 2
- Use the maximum generation length of 15

In [9]:
eos_token = '<eos>'
eos_id = vocab.index(eos_token)
max_gen_length = 15

In [24]:
def greedy_decode(seed_text: str, tokenizer: Tokenizer, max_gen_length = 15):
    """Greedy decodes with seed text.

        Args:
        seed_text: The seed string to be used as initial input to the model.
        tokenizer: The tokenizer for converting word to index and back.

        Your code should do the followings:
          1. Convert current_text to sequences of indices
          2. Predict the next token using the model and choose the token with the highest score as output
          3. Append the predicted index to current_text
          4. Loop until completion
          5. Return text prediction and a list of probabilities of each step

        You do not need to stop early when end-of-sequence token is generated and can continue decoding
        until max_gen_length is reached. We can filter the eos token out later.
    """
    probs = []
    current_text = torch.Tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    is_eos = False

    while len(current_text[0]) < max_gen_length:
        prediction = model(current_text)
        pred_probs = nn.functional.softmax(prediction[0], dim=-1)

        top_probs, top_token_ids = torch.topk(pred_probs, 1)
        top_probs.squeeze_(1)
        top_token_ids.squeeze_(1)
        # print(top_token_ids, top_token_ids.shape)
        # original_text = tokenizer.decode(current_text[0].tolist())
        # top_tokens = tokenizer.decode(top_token_ids.tolist())
        # print(f"text: {original_text}, top_tokens: {top_tokens}, top_token_ids: {top_token_ids}")
        if not is_eos:
            probs.append(top_probs[-1].item())
            
        if top_token_ids[-1].item() == eos_id:
            is_eos = True


        current_text = torch.cat((current_text, torch.tensor([[top_token_ids[-1]]])), dim=1)

    # output = tokenizer.decode(current_text[0].tolist())
    output = current_text[0].tolist()

    return output,probs

In [11]:
def clean_output(token_ids, eos_token_id) -> str:
    """Drop eos_token and every words that follow"""
    # token_ids = tokenizer.encode(text).ids
    # print(token_ids)
    if eos_token_id in token_ids:
        eos_index = token_ids.index(eos_token_id)
        token_ids = token_ids[:eos_index]
    
    # print(token_ids)
    return tokenizer.decode(token_ids)

In [12]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
for seed in sample_seeds:
    output_text, probs = greedy_decode(seed, tokenizer)
    cleaned_text = clean_output(output_text, stoi[eos_token])
    print(f"Seed: {seed}\nOutput: {cleaned_text}\n")
    print(f"probs: {probs}")

Seed: to
Output: to encourage creativity in the new york bill

probs: [0.04071051627397537, 0.26613038778305054, 0.2581597864627838, 0.2311645746231079, 0.07794328778982162, 0.06978005170822144, 0.22445201873779297, 0.06381556391716003]
Seed: america
Output: america s lethal export

probs: [0.44965478777885437, 0.046399738639593124, 0.7206958532333374, 0.9869425296783447]
Seed: people
Output: people to balloon to make a criminal with a dog with a callous rival

probs: [0.35848456621170044, 0.039465807378292084, 0.26952916383743286, 0.033772896975278854, 0.28156718611717224, 0.030896630138158798, 0.3262553811073303, 0.32244089245796204, 0.034891147166490555, 0.2229834347963333, 0.5808281898498535, 0.04674489051103592, 0.2661653459072113, 0.9633656740188599]
Seed: next
Output: next phenom english clubs 2 call another deal in the same arrivals

probs: [0.17242354154586792, 0.23235617578029633, 0.7396268844604492, 0.07888376712799072, 0.04404294118285179, 0.057693492621183395, 0.0329695269

Your output should be:

*   to encourage creativity in the new york bill
*   america s lethal export
*   people to balloon to make a criminal with a dog with a callous rival
*   next phenom english clubs 2 call another deal in the same arrivals
*   picture perfect chapter a spot of view of banning care  
*   on the catwalk in saudi arabia







## 2. TODO: Beam search decode

Another well-known decoding method is beam search decoding that focuses more on the overall sequence score.

Instead of greedily choosing the token with the highest score for each step, beam search decoding expands all possible next tokens and keeps the __k__ most likely sequence at each step, where __k__ is a user-specified beam size. A sequence score is also calculated according user-specified cal_score() function.
The beam with the highest score after the decoding process is done will be the output.

There are a few things that you need to know before implementing a beam search decoder:
- When the eos token is produced, you can stop expanding that beam
- However, the ended beams must be sorted together with active beams
- The decoding ends when every beams are either ended or reached the maximum length, but for this task, you can continue decoding until the max_gen_len is reached
- We usually work with probability in log scale to avoid numerical underflow. You should use np.log(score) before any calculation
- **As probabilities for some classes will be very small, you must add a very small value to the score before taking log e.g np.log(prob + 0.00000001)**

#### Sequence Score
The naive way to calculate the sequence score is to __multiply every token scores__ together. However, doing so will make the decoder prefer shorter sequence as you multiply the sequence score with a value between \[0,1\] for every tokens in the sequence. Thus, we usually normalize the sequence score with its length by calculating its __geometric mean__ instead.

**You should do this in log scale**

In [13]:
def cal_score(scores, length, normalized=False): #cal score for each beam from a list of probs
    log_scores = [np.log(s + 1e-15) for s in scores]
    # formatted_log_scores = [f"{s:.3f}" for s in log_scores]
    # print(f"log_scores: {formatted_log_scores}")
    if normalized:
        score = sum(log_scores) / length
    else:
        score = sum(log_scores)

    return score

In [33]:
# print(cal_score([1, 2, 3, 8], 4, normalized=True))
# print(cal_score([0.1, 0.2, 0.3, 0.4], 4, normalized=True))

# greedy_probs = [0.04071051627397537, 0.26613038778305054, 0.2581597864627838, 0.2311645746231079, 0.07794328778982162, 0.06978005170822144, 0.22445201873779297, 0.06381556391716003]
# score = cal_score(greedy_probs, len(greedy_probs), normalized=True)
# print(len(greedy_probs), score, math.exp(score))

beam_probs = [0.028798477724194527, 0.22056260704994202, 0.0198432058095932, 0.7207849621772766, 0.032541871070861816, 0.09181717038154602, 0.3693373501300812, 0.1407431811094284, 0.8029244542121887, 0.04307405278086662, 1]
# beam_probs = [0.3077567517757416, 0.020585922524333, 0.5763784050941467, 0.7065061926841736, 0.6073137521743774, 0.11878754198551178, 0.3569733500480652, 0.8232592940330505, 0.5624097585678101, 0.05455133691430092, 1] 
# beam_probs = [0.3077567517757416, 0.020585922524333, 0.5763784050941467, 0.7065061926841736, 0.6073137521743774, 0.11878754198551178, 0.3569733500480652, 0.8232592940330505, 0.2889341711997986, 0.10984371602535248, 1]
score = cal_score(beam_probs, len(beam_probs), normalized=True)
print(len(beam_probs), score, math.exp(score))

11 -1.9491536855433231 0.14239453115625528


In [15]:
import copy

def beam_search_decode(seed_text, max_gen_len, tokenizer, beam_size=5, normalized=False):
    """We will do beam search decoing using seed text in this function.

    Output:
    beams: A list of top k beams after the decoding ended, each beam is a list of
      [seed_text, list of scores, length]

    Your code should do the followings:
    1.Loop until max_gen_len is reached.
    2.During each step, loop thorugh each beam and use it to predict the next word.
      If a beam is already ended, continues without expanding.
    3.Sort all hypotheses according to cal_score().
    4.Keep top k hypotheses to be used at the next step.
    """
    # For each beam we will store (generated text, list of scores, and current length, is_finished)
    # Add initial beam
    seed_ids = torch.Tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    # beams = [[[seed_text], [], 0, False]]
    beams = [(seed_ids, [], 0, False)]
    
    for i in range(max_gen_len):
      new_beams = []
      # print(f"---------- Step {i} ----------")
      for beam in beams:
        if beam[-1] == True: # is_finished
          new_beams.append(beam)
          continue
        token_ids, score_list, length, _ = beam
        # text = tokenizer.decode(token_ids[0].tolist())
        # print(f"tokens: {token_ids[0]}\ntext: {text} \nscore: {score_list}, length: {length}")
        prediction = model(token_ids)
        pred_probs = nn.functional.softmax(prediction[0], dim=-1)
        next_token_pred_probs = pred_probs[-1]
        for next_token_id, next_token_prob in enumerate(next_token_pred_probs):
          new_text = torch.cat((token_ids, torch.tensor([[next_token_id]])), dim=1)
          new_length = length + 1

          new_finished = False
          if next_token_id == stoi[eos_token]:
            new_finished = True
          
          new_score = score_list + [next_token_prob.item()]
          new_beams.append((new_text, new_score, new_length, new_finished))

      new_beams.sort(key=lambda x: cal_score(x[1], x[2], normalized), reverse=True)
      # for i in range(5):
      #   text = tokenizer.decode(new_beams[i][0][0].tolist())
      #   print(f"tokens: {new_beams[i][0][0]}\ntext: {text} \nscore: {new_beams[i][1]}, length: {new_beams[i][2]}")
      #   print(f"score: {cal_score(new_beams[i][1], new_beams[i][2], normalized)}")
      # print(f"new_beams len: {len(new_beams)}")
      beams = new_beams[:beam_size]

    return beams

## 3. Generate!
Generate 6 sentences based on the given seed texts.

Decode with the provided seed texts with beam_size 5. Compare the results between greedy, normalized, and unnormalized decoding.

Print the result using greedy decoding and top 2 results each using unnormalized and normalized decoing for each seed text.

Also, print scores of each candidate according to cal_score(). Use normalization for greedy decoding.

In [ ]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
max_gen_length = 10
# sample_seeds = ["to"]
for seed in sample_seeds:
    print("-Greedy-")
    output_text, probs = greedy_decode(seed, tokenizer)
    cleaned_text = clean_output(output_text, stoi[eos_token])
    score = cal_score(probs, len(probs), normalized=True)
    print(f"{cleaned_text}  {round(np.exp(score), 2)}")
    # print(f"probs {probs}")
    print("-Unnormalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=False)
    for beam in beams[:2]:
        output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
        score = cal_score(beam[1], beam[2], normalized=False)
        print(f"{output_text.title()}  {round(np.exp(score), 2)}")
        print(len(output_text.split(" ")), len(beam[1]), beam[1])
    print("-Normalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=True)
    for beam in beams[:2]:
        output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
        score = cal_score(beam[1], beam[2], normalized=True)
        print(f"{output_text.title()}  {round(np.exp(score), 2)}")
        print(len(output_text.split(" ")), len(beam[1]), beam[1])
    print("")

-Greedy-
to encourage creativity in the new york bill  0.12
-Unnormalized-
To Consult Exploring Recipes For New Jersey  0.0
7 7 [0.03152303770184517, 0.30828332901000977, 0.8540560007095337, 0.06086023151874542, 0.1622042953968048, 0.12873302400112152, 0.7070869207382202]
To Consult Exploring Recipes Up The Pacific Northwest  0.0
8 8 [0.03152303770184517, 0.30828332901000977, 0.8540560007095337, 0.06756700575351715, 0.6541436910629272, 0.05161101743578911, 0.24377505481243134, 0.9981531500816345]
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest  0.22
8 8 [0.03152303770184517, 0.30828332901000977, 0.8540560007095337, 0.06756700575351715, 0.6541436910629272, 0.05161101743578911, 0.24377505481243134, 0.9981531500816345]
To Consult Exploring Recipes Up The Least Of The Week  0.19
10 10 [0.03152303770184517, 0.30828332901000977, 0.8540560007095337, 0.06756700575351715, 0.6541436910629272, 0.029132956638932228, 0.3098841607570648, 0.5536085963249207, 0.041552890092134476, 0

In [ ]:
# Picture Korean A Bonanza Of Contemplation Times Of Trump S Son  0.12
prob_1 = [0.028798477724194527, 0.22056260704994202, 0.0198432058095932, 0.7207849621772766, 0.032541871070861816, 0.09181717038154602, 0.3693373501300812, 0.1407431811094284, 0.8029244542121887, 0.04307405278086662]
score = cal_score(prob_1, len(prob_1), normalized=True)
score_with_eos = cal_score(prob_1 + [1], len(prob_1 + [1]), normalized=True)
print(round(np.exp(score), 2), round(np.exp(score_with_eos), 2))

# On The Whole30 Diet Vowing To Eat Smarter Carbs To Be
prob_2 = [0.3077567517757416, 0.020585922524333, 0.5763784050941467, 0.7065061926841736, 0.6073137521743774, 0.11878754198551178, 0.3569733500480652, 0.8232592940330505, 0.2889341711997986, 0.10984371602535248]
score = cal_score(prob_2, len(prob_2), normalized=True)
score_with_eos = cal_score(prob_2 + [1], len(prob_2 + [1]), normalized=True)
print(round(np.exp(score), 2), round(np.exp(score_with_eos), 2))

# On The Whole30 Diet Vowing To Eat Smarter Carbs For Because
prob_3 = [0.3077567517757416, 0.020585922524333, 0.5763784050941467, 0.7065061926841736, 0.6073137521743774, 0.11878754198551178, 0.3569733500480652, 0.8232592940330505, 0.5624097585678101, 0.05455133691430092]
score = cal_score(prob_3, len(prob_3), normalized=True)
score_with_eos = cal_score(prob_3 + [1], len(prob_3 + [1]), normalized=True)
print(round(np.exp(score), 2), round(np.exp(score_with_eos), 2))

# To make the beam search (normalized) score match the answer, we need to add the eos token to the end of the beam search score list.

0.12 0.14
0.27 0.3
0.26 0.3


Your output should be:


```
-Greedy-
to encourage creativity in the new york bill  0.12
-Unnormalized-
To Consult Exploring Recipes For New Jersey 0.00
To Consult Exploring Recipes Up The Pacific Northwest 0.00
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest 0.22
To Consult Exploring Recipes Up The Least Of The Week 0.19

-Greedy-
america s lethal export  0.35
-Unnormalized-
America S Lethal Export 0.01
America S Desert Aisles 0.01
-Normalized-
America S Lethal Export 0.35
America S Desert Aisles 0.30

-Greedy-
people to balloon to make a criminal with a dog with a callous rival  0.16
-Unnormalized-
People To Balloon For A Criminal 0.00
People To Balloon For A Criminal With Trump 0.00
-Normalized-
People To Balloon For A Criminal With A Second Fiddle 0.16
People To Balloon For A Criminal With Trump 0.16

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals  0.15
-Unnormalized-
Next S Blist Revue 0.00
Next Phenom English Clubs 1 A Chance To Be Back 0.00
-Normalized-
Next S Blist Revue 0.23
Next Phenom English Clubs 1 A Chance To Be Back 0.16

-Greedy-
picture perfect chapter a spot of view of banning care  0.09
-Unnormalized-
Picture Perfect Use Coffee 0.00
Picture Korean A Bonanza Of Pancakes 0.00
-Normalized-
Picture Korean A Bonanza Of Contemplation Times Of Trump S Son 0.14
Picture Korean A Bonanza Of Pancakes 0.11

-Greedy-
on the catwalk in saudi arabia  0.25
-Unnormalized-
On The Billboard Chart 0.00
On The Catwalk In Saudi Arabia 0.00
-Normalized-
On The Whole30 Diet Vowing To Eat Smarter Carbs To Be 0.30
On The Whole30 Diet Vowing To Eat Smarter Carbs For Because 0.30

```



# Answer Questions in MyCourseVille!

Use the seed word "usa" to answer questions in MCV.

In [34]:
sample_seeds = ["usa"]
max_gen_length = 10
# sample_seeds = ["to"]
for seed in sample_seeds:
    print("-Greedy-")
    output_text, probs = greedy_decode(seed, tokenizer)
    cleaned_text = clean_output(output_text, stoi[eos_token])
    score = cal_score(probs, len(probs), normalized=True)
    print(f"{cleaned_text}  {round(np.exp(score), 2)}")
    # print(f"probs {probs}")
    print("-Unnormalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=False)
    for beam in beams[:2]:
        output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
        # remove last prob (eos prob), also minus it from length
        score = cal_score(beam[1], beam[2], normalized=False)
        print(f"{output_text.title()}  {round(np.exp(score), 2)}")
        print(len(output_text.split(" ")), len(beam[1]), beam[1])
    print("-Normalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=True)
    for beam in beams[:2]:
        output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
        score = cal_score(beam[1], beam[2], normalized=True)
        print(f"{output_text.title()}  {round(np.exp(score), 2)}")
        print(len(output_text.split(" ")), len(beam[1]), beam[1])
    print("")

-Greedy-
usa s duty to investigate  0.15
-Unnormalized-
Usa S Duty To Investigate  0.0
5 5 [0.15032975375652313, 0.01845468021929264, 0.47833025455474854, 0.05866711959242821, 0.8444095253944397]
Usa S Bleak Season 3 Episode 4 Recap Chicken  0.0
9 9 [0.15032975375652313, 0.01804458536207676, 0.38699889183044434, 0.18051539361476898, 0.8674392104148865, 0.2880602478981018, 0.48781028389930725, 0.08663538098335266, 0.09267482906579971]
-Normalized-
Usa S Bleak Season 1 Episode 2 Darkness Descends  0.23
9 9 [0.15032975375652313, 0.01804458536207676, 0.38699889183044434, 0.2305964231491089, 0.8998907804489136, 0.21810458600521088, 0.07934974879026413, 0.47243404388427734, 0.9940153956413269]
Usa S Bleak Season 3 Episode 4 Recap A Seduction  0.21
10 10 [0.15032975375652313, 0.01804458536207676, 0.38699889183044434, 0.18051539361476898, 0.8674392104148865, 0.2880602478981018, 0.48781028389930725, 0.17257606983184814, 0.04379192739725113, 0.9871757626533508]

